In [11]:
import win32api as wapi
import time
import win32con

keyList = ['\b']
for char in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ 1234567890':
    keyList.append(char)

def get_keys():
    keys = []
    for key in keyList:
        if wapi.GetAsyncKeyState(ord(key)):
            keys.append(key)

    if wapi.GetAsyncKeyState(win32con.VK_MENU):
        keys.append('ALT')        
    if wapi.GetAsyncKeyState(win32con.VK_LEFT):
        keys.append('LEFT')
    if wapi.GetAsyncKeyState(win32con.VK_RIGHT):
        keys.append('RIGHT')
    if wapi.GetAsyncKeyState(win32con.VK_UP):
        keys.append('UP')
    return keys

In [24]:
import time
import numpy as np
from threading import Thread
from pynput.mouse import Listener
from functools import partial
import os
import win32api

save_batch_size = 2000 # 2000 - save every minute
data_path = 'data'


isPressed = False
def on_click(x, y, button, pressed):
    global isPressed
    isPressed = pressed
    
def detect_clicks():
    with Listener(on_click=partial(on_click)) as listener:
        listener.join()

        
if not os.path.exists(data_path):
    os.mkdir(data_path)

# New thread for click detecting
thread = Thread(target=detect_clicks)
thread.start()

print('Recording mouse and keyboard...')
while True:
    data = np.zeros((save_batch_size, 11))
    for i in range(save_batch_size):
        keys = get_keys()
        x, y = win32api.GetCursorPos()
        data[i] = np.array([x, 
                            y, 
                            isPressed,
                            *[key in keys for key in ['Q', 'W', 'E', ' ', 'UP', 'RIGHT', 'LEFT']],
                            time.time()])
        print(data[i])
        time.sleep(1)
        time.sleep(0.033) # 30 FPS
    path = os.path.join(data_path, f'mouse-keys-{int(data[0][7])}-{int(data[-1][7])}.npy')
    np.save(path, data)
    print('Saved:', path)

Recording mouse and keaboard...
[8.21000000e+02 5.63000000e+02 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.00000000e+00 0.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.56477587e+09]
[7.49000000e+02 5.86000000e+02 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.56477587e+09]
[7.49000000e+02 5.86000000e+02 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.00000000e+00 1.56477587e+09]
[7.49000000e+02 5.86000000e+02 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.00000000e+00 1.56477587e+09]
[7.49000000e+02 5.86000000e+02 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.00000000e+00 1.00000000e+00 1.56477587e+09]
[7.49000000e+02 5.86000000e+02 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 

KeyboardInterrupt: 